In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,...,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,target
0,2020,1,1,1,3,2020-01-01,N951WN,ONT,"Ontario, CA",SFO,...,2020-01-01 18:00:00,12.2,-7.3,25.0,0.0,330.0,18.4,1017.3,3.0,1
1,2020,1,1,1,3,2020-01-01,N467WN,ONT,"Ontario, CA",SFO,...,2020-01-01 11:00:00,3.9,-3.4,59.0,0.0,210.0,5.4,1017.6,2.0,0
2,2020,1,1,1,3,2020-01-01,N7885A,ONT,"Ontario, CA",SJC,...,2020-01-01 20:00:00,12.8,-7.3,24.0,0.0,290.0,11.2,1015.8,2.0,0


In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [9]:
# getting dummu values for categorical columns
QUARTER_dummies = pd.get_dummies(flight_data['QUARTER'], prefix='QUARTER', drop_first=True)
MONTH_dummies = pd.get_dummies(flight_data['MONTH'], prefix='MONTH', drop_first=True)
DAY_OF_MONTH_dummies = pd.get_dummies(flight_data['DAY_OF_MONTH'], prefix='DAY_OF_MONTH', drop_first=True)
DAY_OF_WEEK_dummies = pd.get_dummies(flight_data['DAY_OF_WEEK'], prefix='DAY_OF_WEEK', drop_first=True)
Aircraft_type_dummies = pd.get_dummies(flight_data['aircraft_type_new'], prefix='Aircraft_type', drop_first=True)

In [10]:
# Aircraft_type_dummies.columns = ['Aircraft_{}'.format(i) for i in range(len(Aircraft_type_dummies.columns))]

In [11]:
flight_data = flight_data.drop(l_categorical_cols, axis=1)

In [12]:
dummy_flight_data = pd.concat([flight_data, QUARTER_dummies, MONTH_dummies,
                               DAY_OF_MONTH_dummies, DAY_OF_WEEK_dummies, Aircraft_type_dummies ], axis=1)

In [13]:
del Aircraft_type_dummies,QUARTER_dummies,MONTH_dummies,DAY_OF_MONTH_dummies,DAY_OF_WEEK_dummies

In [14]:
train = dummy_flight_data[dummy_flight_data['YEAR'] == 2020]
test = dummy_flight_data[dummy_flight_data['YEAR'] == 2019]

In [15]:
del flight_data, dummy_flight_data

In [16]:
del train

In [2]:
train = pd.read_csv('undersampled_training_data_2020.csv')

In [18]:
logistic_regression = LogisticRegression(multi_class="ovr")
logistic_regression.fit(
    train.drop(["target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

LogisticRegression(multi_class='ovr')

In [19]:
path = "./models"
filename = "2020train_LR_ovr_undersample.mdl"

In [20]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression, f, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
logistic_regression = pickle.load(open("./models/2020train_LR_ovr_undersample.mdl",'rb'))

C:\anaconda\envs\python_37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [4]:
feature_names = train.columns.drop('target')

In [5]:
feature_importance = abs(logistic_regression.coef_[0])
feature_importance = 100.0 * (feature_importance / feature_importance.max())

In [6]:
importances = pd.DataFrame(
  pd.Series(dict(zip(feature_names, feature_importance)))
).assign(
  normalized=lambda df: df / df.sum()
).sort_values(
  by="normalized", ascending=False
)

In [7]:
importances[["normalized"]].head(10)

,normalized
Aircraft_type_Boeing 737,0.106235
Aircraft_type_Mitsubishi CRJ,0.071688
MONTH_2,0.064768
QUARTER_4,0.057562
QUARTER_2,0.049770
prcp,0.047365
MONTH_11,0.033286
DEP_HOUR,0.031761
Aircraft_type_Airbus A320,0.031456
DAY_OF_MONTH_13,0.025300


In [21]:
# predicting the test results 
y_pred = logistic_regression.predict(test.drop(["YEAR","target"], axis=1).values)

In [22]:
print(confusion_matrix(test[target],y_pred))

[[4166272  777252  599748  956811]
 [ 445517  156211  135873  166097]
 [ 130957   44667   64198   78121]
 [  84785   24700   47410   58844]]


In [23]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.86      0.64      0.74   6500083
           1       0.16      0.17      0.16    903698
           2       0.08      0.20      0.11    317943
           3       0.05      0.27      0.08    215739

    accuracy                           0.56   7937463
   macro avg       0.29      0.32      0.27   7937463
weighted avg       0.73      0.56      0.63   7937463



In [24]:
print("Testing  set score for model: %f" % logistic_regression.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.560069


In [26]:
###### logistic regression multinomial ####
logistic_regression_mul = LogisticRegression(multi_class="multinomial")
logistic_regression_mul.fit(
    train.drop(["target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='multinomial')

In [27]:
path = "./models"
filename = "2020train_LR_mul_undersample.mdl"

In [28]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression_mul, f, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
logistic_regression_mul = pickle.load(open("./models/2020train_LR_mul.mdl",'rb'))

C:\anaconda\envs\python_37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [9]:
feature_importance_mul = abs(logistic_regression_mul.coef_[0])
feature_importance_mul = 100.0 * (feature_importance_mul / feature_importance_mul.max())

In [10]:
importances_mul = pd.DataFrame(
  pd.Series(dict(zip(feature_names, feature_importance_mul)))
).assign(
  normalized=lambda df: df / df.sum()
).sort_values(
  by="normalized", ascending=False
)

In [11]:
importances_mul[["normalized"]].head(10)

,normalized
DEP_HOUR,0.375379
coco,0.114452
wspd,0.060758
temp,0.055337
dwpt,0.045036
prcp,0.029117
MONTH_2,0.026683
rhum,0.021202
QUARTER_4,0.020269
QUARTER_2,0.020048


In [29]:
# predicting the test results 
y_pred = logistic_regression_mul.predict(test.drop(["YEAR","target"], axis=1).values)

In [30]:
print(confusion_matrix(test[target],y_pred))

[[4040892  532328 1173191  753672]
 [ 396318   79578  264848  162954]
 [ 119154   23248  113570   61971]
 [  79927   13836   78824   43152]]


In [31]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.87      0.62      0.73   6500083
           1       0.12      0.09      0.10    903698
           2       0.07      0.36      0.12    317943
           3       0.04      0.20      0.07    215739

    accuracy                           0.54   7937463
   macro avg       0.28      0.32      0.25   7937463
weighted avg       0.73      0.54      0.61   7937463



In [32]:
print("Testing  set score for model: %f" % logistic_regression_mul.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.538861
